In [1]:
%%capture
# !pip install unsloth
# # Also get the latest nightly Unsloth!
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

In [10]:
# %pip uninstall unsloth unsloth_zoo -y
# %pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# %pip install --upgrade --no-cache-dir "git+https://github.com/unslothai/unsloth-zoo.git"

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [2]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/Meta-Llama-3.1-8B",
    model_name = "unsloth/Meta-Llama-3.1-8B",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-02-03 15:34:36.643539: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-03 15:34:36.657643: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1738564476.674736 2751950 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1738564476.679759 2751950 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-03 15:34:36.697365: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Unsloth: OpenAI failed to import - ignoring for now.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [3]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                     "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0.05, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2025.1.8 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
# from datasets import load_dataset
# from datasets import Dataset, concatenate_datasets

# import pandas as pd

# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# def formatting_prompts_func(examples):
#     instructions = examples["instruction"]
#     inputs = examples["input"]
#     outputs = examples["output"]
#     texts = []
#     for instruction, input_text, output in zip(instructions, inputs, outputs):
#         text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
#         texts.append(text)
#     return { "text": texts }

# dataset = load_dataset("passionMan/diabetes_v9", split="train")
# dataset = dataset.map(formatting_prompts_func, batched=True)

# dataset_df = pd.DataFrame(dataset)

# long_data = dataset_df[dataset_df['dataset'].isin(['pubmedqa', 'icliniq', 'pubmed', 'chemdner', 'medal', 'mfc'])].reset_index(drop=True)
# short_data = dataset_df[~dataset_df['dataset'].isin(['pubmedqa', 'icliniq', 'pubmed', 'chemdner', 'medal', 'mfc'])].reset_index(drop=True)

# long_dataset = Dataset.from_pandas(long_data)
# short_dataset = Dataset.from_pandas(short_data)

# combined_dataset = concatenate_datasets([short_dataset, long_dataset])


In [5]:
# from datasets import load_dataset
# from datasets import Dataset, concatenate_datasets

# import pandas as pd

# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# def formatting_prompts_func(examples):
#     instructions = examples["instruction"]
#     inputs = examples["input"]
#     outputs = examples["output"]
#     texts = []
    
#     for instruction, input_text, output in zip(instructions, inputs, outputs):
#         text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
#         texts.append(text)

#     return { "text": texts }

# dataset = load_dataset("passionMan/diabetes_v11", split="train")
# dataset = dataset.map(formatting_prompts_func, batched=True)

# dataset_df = pd.DataFrame(dataset)

# long_data = dataset_df[dataset_df['dataset'].isin(['meddialog', 'pubmed', 'ddi'])].reset_index(drop=True)
# short_data = dataset_df[~dataset_df['dataset'].isin(['meddialog', 'pubmed', 'ddi'])].reset_index(drop=True)

# def tokenize_and_set_max_length(examples):
#     max_len = 4096 if examples["dataset"] in ['meddialog', 'pubmed', 'ddi'] else 1024
#     return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_len)

# long_dataset = Dataset.from_pandas(long_data).map(tokenize_and_set_max_length, batched=True)
# short_dataset = Dataset.from_pandas(short_data).map(tokenize_and_set_max_length, batched=True)

# combined_dataset = concatenate_datasets([short_dataset, long_dataset])

In [6]:
# from datasets import load_dataset, Dataset
# import pandas as pd

# # Alpaca 포맷 적용 템플릿
# alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

# ### Instruction:
# {}

# ### Input:
# {}

# ### Response:
# {}"""

# EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

# # 1️⃣ 데이터셋 로드 및 포맷 적용
# def formatting_prompts_func(examples):
#     texts = [
#         alpaca_prompt.format(inst, inp, out) + EOS_TOKEN
#         for inst, inp, out in zip(examples["instruction"], examples["input"], examples["output"])
#     ]
#     return {"text": texts, "task": examples["task"]}  # task 정보도 유지

# dataset = load_dataset("passionMan/diabetes_v11", split="train")
# dataset = dataset.map(formatting_prompts_func, batched=True)

# # 2️⃣ Task별 max_length 설정
# task_max_length = {
#     "alternative_diet": 1024,
#     "daily_diets": 512,
#     "dfh_info": 512,
#     "generation": 3072,
#     "ie": 2048,
#     "nli": 2048,
#     "qa_objective_1": 2048,
#     "qa_objective_2": 1024,
#     "qa_objective_3": 2048,
#     "re": 2048,
#     "summarization": 3072,  # 가장 긴 평균 길이 적용
# }

# # 3️⃣ Tokenization 및 max_length 적용 (batched=True 대응)
# def tokenize_and_set_max_length(examples):
#     # 각 샘플별 max_length 설정 후 개별적으로 토큰화 적용
#     tokenized_texts = [
#         tokenizer(
#             text,
#             truncation=True,
#             padding="max_length",
#             max_length=task_max_length.get(task, 1024)  # 기본값 1024
#         ) for text, task in zip(examples["text"], examples["task"])
#     ]
    
#     # 토큰화된 결과를 리스트에서 딕셔너리 형태로 변환
#     return {key: [dic[key] for dic in tokenized_texts] for key in tokenized_texts[0]}

# # 🔹 Dataset.map()을 batched=True로 변경하여 처리
# tokenized_dataset = dataset.map(tokenize_and_set_max_length, batched=True, batch_size=4)


In [7]:
from datasets import load_dataset
from datasets import Dataset, concatenate_datasets

import pandas as pd

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []
    
    for instruction, input_text, output in zip(instructions, inputs, outputs):
        text = alpaca_prompt.format(instruction, input_text, output) + EOS_TOKEN
        texts.append(text)

    return { "text": texts }

dataset = load_dataset("passionMan/diabetes_v13", split="train")
dataset = dataset.map(formatting_prompts_func, batched=True)

dataset_df = pd.DataFrame(dataset)

long_data = dataset_df[dataset_df['dataset'].isin(['pubmed', 'bionli', 'chemdner'])].reset_index(drop=True)
medium_data = dataset_df[dataset_df['dataset'].isin(['ddi', 'pubmedqa', 'medqa', 'meddialog'])].reset_index(drop=True)
short_data = dataset_df[~dataset_df['dataset'].isin(['medmcqa', 'diabetes_food_hub'])].reset_index(drop=True)

def tokenize_and_set_max_length(examples):
    if examples["dataset"] in ["pubmed", 'bionli', 'chemdner']:
        max_len = 4096
    elif examples["dataset"] in ["ddi", "pubmedqa", "medqa", "meddialog"]:
        max_len = 2048
    else:
        max_len = 1024
    return tokenizer(examples["text"], truncation=True, padding="max_length", max_length=max_len)

long_dataset = Dataset.from_pandas(long_data).map(tokenize_and_set_max_length, batched=True)
medium_dataset = Dataset.from_pandas(medium_data).map(tokenize_and_set_max_length, batched=True)
short_dataset = Dataset.from_pandas(short_data).map(tokenize_and_set_max_length, batched=True)

combined_dataset = concatenate_datasets([short_dataset, medium_dataset, long_dataset])

README.md:   0%|          | 0.00/648 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/47.5M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/4.80M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/41622 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/6044 [00:00<?, ? examples/s]

Map:   0%|          | 0/41622 [00:00<?, ? examples/s]

Map:   0%|          | 0/8905 [00:00<?, ? examples/s]

Map:   0%|          | 0/8672 [00:00<?, ? examples/s]

Map:   0%|          | 0/17577 [00:00<?, ? examples/s]

In [8]:
# from trl import SFTTrainer
# from transformers import TrainingArguments
# from unsloth import is_bfloat16_supported
# from datasets import Dataset, concatenate_datasets
# import pandas as pd
# import numpy as np


# def compute_weights(example):
#     return {"weight": 0.4 if example['dataset'] in ['pubmedqa', 'icliniq', 'pubmed', 'chemdner', 'medal', 'mfc'] else 0.6}

# weighted_dataset = combined_dataset.map(compute_weights)

# weights = np.array(weighted_dataset["weight"])
# weights = weights / weights.sum()  

# weighted_indices = np.random.choice(len(combined_dataset), size=len(combined_dataset), p=weights)
# resampled_dataset = combined_dataset.select(weighted_indices)

# if isinstance(resampled_dataset, dict):
#     resampled_dataset = Dataset.from_dict(resampled_dataset)

# if "text" not in resampled_dataset.column_names:
#     resampled_dataset = resampled_dataset.map(lambda x: {"text": x["input"] + " " + x["output"]})

# trainer = SFTTrainer(
#     model=model,
#     tokenizer=tokenizer,
#     train_dataset=resampled_dataset,
#     dataset_text_field="text",
#     dataset_num_proc=16, 

#     args=TrainingArguments(
#         per_device_train_batch_size=32, 
#         gradient_accumulation_steps=1,  
#         warmup_steps=300,  
#         num_train_epochs=3, 
#         learning_rate=3e-5,  
#         fp16=not is_bfloat16_supported(),
#         bf16=is_bfloat16_supported(),  
#         optim="adamw_8bit",  
#         weight_decay=0.01,  
#         lr_scheduler_type="linear",  
#         logging_steps=100,  
#         seed=3407,  
#         save_steps=200,  
#         output_dir="outputs/weighted_sampling9", 
#         report_to="none",  
#     ),
# )

In [9]:
# from datasets import Dataset, concatenate_datasets
# import numpy as np
# from trl import SFTTrainer
# from transformers import TrainingArguments
# from unsloth import is_bfloat16_supported
# import pandas as pd

# def compute_weights(example):
#     if example['dataset'] in ['daily_diets']:
#         return {"weight": 0.3}
#     elif example['dataset'] in ['alternative_diet', 'dfh_info']:
#         return {"weight": 0.5}
#     elif example['dataset'] in ['meddialog', 'pubmed']:
#         return {"weight": 0.7}
#     elif example['dataset'] in ['chemdner']:
#         return {"weight": 1.2} 
#     elif example['dataset'] in ['ddi']:
#         return {"weight": 2.0}
#     elif example['dataset'] in ['bionli', 'chemdner']:
#         return {"weight": 3.0}
#     else:
#         return {"weight": 1.0}
    

# if "dataset" in combined_dataset.column_names:
#     weighted_dataset = combined_dataset.map(compute_weights)
# else:
#     print("Warning: 'dataset' 컬럼이 존재하지 않습니다. 가중치 계산이 불가능할 수 있습니다.")

# weights = np.array(weighted_dataset["weight"])
# weights = weights / weights.sum() 

# weighted_indices = np.random.choice(len(combined_dataset), size=len(combined_dataset), p=weights)

# if "__index_level_0__" in combined_dataset.column_names:
#     combined_dataset = combined_dataset.remove_columns(["__index_level_0__"])

# resampled_dataset = combined_dataset.select(weighted_indices)

# if "text" not in resampled_dataset.column_names:
#     resampled_dataset = resampled_dataset.map(lambda x: {"text": x["input"] + " " + x["output"]})

# trainer = SFTTrainer(
#     model=model,
#     tokenizer=tokenizer,
#     train_dataset=resampled_dataset,
#     dataset_text_field="text",
#     dataset_num_proc=16, 

#     args=TrainingArguments(
#         per_device_train_batch_size=32, 
#         gradient_accumulation_steps=1,  
#         warmup_steps=300,  
#         num_train_epochs=3, 
#         learning_rate=3e-5,  
#         fp16=not is_bfloat16_supported(),
#         bf16=is_bfloat16_supported(),  
#         optim="adamw_8bit",  
#         weight_decay=0.01,  
#         lr_scheduler_type="linear",  
#         logging_steps=100,  
#         seed=3407,  
#         save_steps=200,  
#         output_dir="outputs/weighted_sampling11", 
#         report_to="none",  
#     ),
# )


In [10]:
# from datasets import Dataset
# import numpy as np
# from trl import SFTTrainer
# from transformers import TrainingArguments
# from unsloth import is_bfloat16_supported
# import pandas as pd

# # 📌 1️⃣ Task별 가중치 설정 (이미지 속 데이터 분포 반영)
# def compute_weights(example):
#     task_weights = {
#         "daily_diets": 0.4, # 3000
#         "alternative_diet": 0.5, # 3000
#         "dfh_info": 0.6, # 3000
#         "summarization": 4.0, 
#         "generation": 0.5,
#         "nli": 0.9,
#         "qa_objective_3": 1.0,
#         "qa_objective_1": 1.0,
#         "qa_objective_2": 1.0,
#         "re": 1.5,
#         "ie": 1.0,
#     }
#     return {"weight": task_weights.get(example["task"], 1.0)}

# weighted_dataset = tokenized_dataset.map(compute_weights)

# weights = np.array(weighted_dataset["weight"])
# weights = weights / weights.sum() 

# weighted_indices = np.random.choice(len(tokenized_dataset), size=len(tokenized_dataset), p=weights)
# resampled_dataset = tokenized_dataset.select(weighted_indices)
# resampled_dataset = weighted_dataset.shuffle(seed=3407).select(range(len(weighted_dataset)))

# trainer = SFTTrainer(
#     model=model,
#     tokenizer=tokenizer,
#     train_dataset=resampled_dataset,
#     dataset_text_field="text",
#     dataset_num_proc=8, 

#     args=TrainingArguments(
#         per_device_train_batch_size=32, 
#         gradient_accumulation_steps=2,  
#         warmup_steps=300,  
#         num_train_epochs=3, 
#         learning_rate=3e-5,  
#         fp16=not is_bfloat16_supported(),
#         bf16=is_bfloat16_supported(),  
#         optim="adamw_8bit",  
#         weight_decay=0.01,  
#         lr_scheduler_type="linear",  
#         logging_steps=100,  
#         seed=3407,  
#         save_steps=200,  
#         output_dir="outputs/weighted_sampling12", 
#         report_to="none",  
#     ),
# )


In [11]:
from datasets import Dataset, concatenate_datasets
import numpy as np
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import pandas as pd

def compute_weights(example):
    if example['task'] in ['daily_diets']:
        return {"weight": 0.3}
    elif example['task'] in ['alternative_diet']:
        return {"weight": 0.5}
    elif example['task'] in ['dfh_info']:
        return {"weight": 0.7}
    elif example['task'] in ['summarization', 'generation']:
        return {"weight": 0.8}
    elif example['task'] in ['ie', 'nli', 'qa_objective_1', 'qa_objective_2', 'qa_objective_3']:
        return {"weight": 1.0} 
    elif example['dataset'] in ['ddi']:
        return {"weight": 2.0}
    else:
        return {"weight": 1.0}
    

if "dataset" in combined_dataset.column_names:
    weighted_dataset = combined_dataset.map(compute_weights)
else:
    print("Warning: 'dataset' 컬럼이 존재하지 않습니다. 가중치 계산이 불가능할 수 있습니다.")

weights = np.array(weighted_dataset["weight"])
weights = weights / weights.sum() 

weighted_indices = np.random.choice(len(combined_dataset), size=len(combined_dataset), p=weights)

if "__index_level_0__" in combined_dataset.column_names:
    combined_dataset = combined_dataset.remove_columns(["__index_level_0__"])

resampled_dataset = combined_dataset.select(weighted_indices)

if "text" not in resampled_dataset.column_names:
    resampled_dataset = resampled_dataset.map(lambda x: {"text": x["input"] + " " + x["output"]})

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=resampled_dataset,
    dataset_text_field="text",
    dataset_num_proc=16, 

    args=TrainingArguments(
        per_device_train_batch_size=32, 
        gradient_accumulation_steps=1,  
        warmup_steps=300,  
        num_train_epochs=3, 
        learning_rate=3e-5,  
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),  
        optim="adamw_8bit",  
        weight_decay=0.01,  
        lr_scheduler_type="linear",  
        logging_steps=100,  
        seed=3407,  
        save_steps=200,  
        output_dir="outputs/weighted_sampling_v4", 
        report_to="none",  
    ),
)


Map:   0%|          | 0/35154 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [12]:
trainer_stats = trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 35,154 | Num Epochs = 3
O^O/ \_/ \    Batch size per device = 32 | Gradient Accumulation steps = 1
\        /    Total batch size = 32 | Total steps = 3,297
 "-____-"     Number of trainable parameters = 20,971,520
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process

Step,Training Loss
100,1.879900
200,1.459000
300,1.311400
400,1.298100
500,1.282400
600,1.296100
700,1.263700
800,1.253100
900,1.247000
1000,1.232800


/data/jaesung/anaconda3/envs/meal/lib/python3.12/site-packages/peft/utils/other.py:689: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 941075e6-9927-473c-b87d-b5bde650f48f)') - silently ignoring the lookup for the file config.json in unsloth/meta-llama-3.1-8b-bnb-4bit.
  warnings.warn(
/data/jaesung/anaconda3/envs/meal/lib/python3.12/site-packages/peft/utils/save_and_load.py:243: UserWarning: Could not find a config file in unsloth/meta-llama-3.1-8b-bnb-4bit - will assume that the vocabulary was not modified.
  warnings.warn(


: 

In [15]:
## inference

In [ ]:
from datasets import Dataset, concatenate_datasets
import numpy as np
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
import pandas as pd

def compute_weights(example):
    if example['task'] in ['daily_diets']:
        return {"weight": 0.3}
    elif example['task'] in ['generation', 'alternative_diet']:
        return {"weight": 0.5}
    elif example['task'] in ['dfh_info', 'nli', 'summarization']:
        return {"weight": 0.7}
    elif example['task'] in ['ie', 'qa_objective_1', 'qa_objective_2', 'qa_objective_3']:
        return {"weight": 1.0} 
    elif example['dataset'] in ['ddi']:
        return {"weight": 2.0}
    else:
        return {"weight": 1.0}
    

if "dataset" in combined_dataset.column_names:
    weighted_dataset = combined_dataset.map(compute_weights)
else:
    print("Warning: 'dataset' 컬럼이 존재하지 않습니다. 가중치 계산이 불가능할 수 있습니다.")

weights = np.array(weighted_dataset["weight"])
weights = weights / weights.sum() 

weighted_indices = np.random.choice(len(combined_dataset), size=len(combined_dataset), p=weights)

if "__index_level_0__" in combined_dataset.column_names:
    combined_dataset = combined_dataset.remove_columns(["__index_level_0__"])

resampled_dataset = combined_dataset.select(weighted_indices)

if "text" not in resampled_dataset.column_names:
    resampled_dataset = resampled_dataset.map(lambda x: {"text": x["input"] + " " + x["output"]})

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=resampled_dataset,
    dataset_text_field="text",
    dataset_num_proc=16, 

    args=TrainingArguments(
        per_device_train_batch_size=64, 
        gradient_accumulation_steps=1,  
        warmup_steps=300,  
        num_train_epochs=3, 
        learning_rate=6e-5,  
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),  
        optim="adamw_8bit",  
        weight_decay=0.01,  
        lr_scheduler_type="linear",  
        logging_steps=100,  
        seed=3407,  
        save_steps=200,  
        output_dir="outputs/weighted_sampling_v1", 
        report_to="none",  
    ),
)


Map:   0%|          | 0/43179 [00:00<?, ? examples/s]

In [16]:

from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "/data/jaesung/llm_for_diabetes/src/model/outputs/weighted_sampling11/checkpoint-3453",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2025.1.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA A100-PCIE-40GB. Max memory: 39.394 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.0+cu124. CUDA: 8.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post2. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [13]:


FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""


inputs = tokenizer(
[
    alpaca_prompt.format(
        "Recommend a daily diet that includes a specific ingredient.",

        "Create a diet that includes baby bok choy(roots trimmed and roughly chopped).",

        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 2048)

<|begin_of_text|>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Recommend a daily diet that includes a specific ingredient.

### Input:
Create a diet that includes baby bok choy(roots trimmed and roughly chopped).

### Response:
{"Breakfast": "Ginger Infused Oatmeal", "Lunch": "Turkey Meatball \u201cWonton\u201d Soup with Bok Choy & Carrots", "Dinner": "Roasted Sweet Potatoes with Lemon-Dill Yogurt Sauce"}<|end_of_text|>


In [14]:
import random
import numpy as np
import pandas as pd  # pandas 임포트 추가
from datasets import load_dataset, Dataset, DatasetDict

# 시드 값 설정
SEED = 42

# Python의 random 모듈 시드 고정
random.seed(SEED)

# NumPy 시드 고정
np.random.seed(SEED)

# 1. 데이터셋 로드
dataset_name = "passionMan/diabetes_v11"
train_dataset = load_dataset(dataset_name, split="train")
test_dataset = load_dataset(dataset_name, split="test")

# 2. 중복 확인을 위한 train 데이터의 (input, output) 세트 생성
train_pairs = set((row["input"], row["output"]) for row in train_dataset)

# 3. 각 task별 샘플 개수 설정
task_sample_limits = {
    'qa_objective_1': 50,
    'qa_objective_2': 50,
    'qa_objective_3': 50,
    'nli': 50,
    're': 50,
    'ie': 50,
    "summarization": 50,
    "generation": 50,
    'alternative_diet': 50,
    'daily_diets': 50,
}
default_sample_limit = 50  # 나머지 task는 100개씩 샘플링

# 4. task별 샘플링 데이터 초기화
task_sampled_data = {}
# task_counts를 task_sample_limits로 초기화
task_counts = {task: 0 for task in task_sample_limits}

# 5. Task별 샘플링
# 테스트 데이터셋을 셔플링하여 랜덤 샘플링 보장
test_dataset = test_dataset.shuffle(seed=SEED)

for row in test_dataset:
    task = row["task"]
    dataset_value = row.get("dataset")  # dataset 컬럼 값 확인
    input_output_pair = (row["input"], row["output"])

    # qa_objective 세분화
    if task == "qa_objective":
        if dataset_value == "medqa":
            task = "qa_objective_1"  # medqa로 세분화
        elif dataset_value == "medmcqa":
            task = "qa_objective_2"  # medmcqa로 세분화
        else:
            continue  # medqa, medmcqa가 아니면 건너뛰기

    # 해당 task의 샘플링 제한 확인
    sample_limit = task_sample_limits.get(task, default_sample_limit)

    # 샘플링 조건 확인
    if task_counts.get(task, 0) < sample_limit and input_output_pair not in train_pairs:
        if task not in task_sampled_data:
            task_sampled_data[task] = []
        task_sampled_data[task].append(row)
        task_counts[task] += 1

    # 모든 task의 샘플링이 완료되면 종료
    if all(task_counts.get(task, 0) >= task_sample_limits.get(task, default_sample_limit) for task in task_sample_limits):
        break

# 6. 결과 출력 및 확인
for task, samples in task_sampled_data.items():
    print(f"Task: {task}, Sampled: {len(samples)}")
    for sample in samples[:5]:  # 첫 5개 샘플만 출력
        print(sample)

# 7. 필요 시 샘플링된 데이터 저장
# pandas를 사용하여 리스트의 딕셔너리를 데이터프레임으로 변환 후 Dataset으로 변환
sampled_dataset = DatasetDict({
    task: Dataset.from_pandas(pd.DataFrame(samples)) for task, samples in task_sampled_data.items()
})

# 저장 (필요시 주석 제거)
# sampled_dataset.save_to_disk("sampled_test_dataset")


Task: daily_diets, Sampled: 50
{'dataset': 'diabetes_food_hub', 'split_data': 'test', 'task': 'daily_diets', 'instruction': 'Recommend a daily diet that includes a specific ingredient.', 'input': 'Create a diet that includes Splenda® Monk Fruit Granulated Sweetener(for the BBQ sauce).', 'output': '{"Breakfast": "Masala Omelette with Mixed Veggies", "Lunch": "Air Fryer \\u201cHoney\\u201d BBQ Cauliflower Wings", "Dinner": "Turkey Meatball \\u201cWonton\\u201d Soup with Bok Choy & Carrots"}', 'text_length': 317, '__index_level_0__': 48548}
{'dataset': 'diabetes_food_hub', 'split_data': 'test', 'task': 'daily_diets', 'instruction': 'Analyze and summarize the nutritional content of a daily diet.', 'input': '{"Breakfast": "Greek Vegetable Pita Pizzas", "Lunch": "Peach & Cream Smoothie", "Dinner": "Turkey Meatball \\u201cWonton\\u201d Soup with Bok Choy & Carrots"}', 'output': '{"Daily Total": {"Calories": 670.0, "Carbohydrate (g)": 71.0, "Fiber (g)": 12.0, "Protein (g)": 49.0, "Fat (g)": 25

In [11]:
import random
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

# ✅ 시드 설정
SEED = 42
random.seed(SEED)
np.random.seed(SEED)

# ✅ 1. 데이터셋 로드
dataset_name = "passionMan/diabetes_v11"
train_dataset = load_dataset(dataset_name, split="train")
test_dataset = load_dataset(dataset_name, split="test")

# ✅ 2. 중복 방지를 위한 (input, output) 세트 생성
train_pairs = set((row["input"], row["output"]) for row in train_dataset)

# ✅ 3. `qa`, `ner`, `nli`, `ned`만 샘플링 대상
task_sample_limits = {
    'qa_objective_1': 50,  # medqa
    'qa_objective_2': 50,  # medmcqa
    'qa_objective_3': 50,  # pubmedqa
    'nli': 50,
    'ie': 50,
    're': 50,
}
default_sample_limit = 50  # 기본값 50개 샘플링

# ✅ 4. 샘플링 데이터 저장용 딕셔너리 초기화
task_sampled_data = {}
task_counts = {task: 0 for task in task_sample_limits}  # ✅ 필요 task만 초기화

# ✅ 5. Task별 샘플링 수행
test_dataset = test_dataset.shuffle(seed=SEED)

for row in test_dataset:
    task = row["task"]
    dataset_value = row.get("dataset")  # ✅ dataset 컬럼 값 확인
    input_output_pair = (row["input"], row["output"])

    # ✅ 5-1. `qa_objective`를 `medqa`와 `medmcqa`로 세분화
    if task == "qa_objective":
        if dataset_value == "medqa":
            task = "qa_objective_1"  # medqa
        elif dataset_value == "medmcqa":
            task = "qa_objective_2"  # medmcqa
        elif dataset_value == "pubmedqa":
            task = "qa_objective_3"  # pubmedqa
        else:
            continue  # ✅ `medqa`, `medmcqa`, `pubmedqa`가 아니면 건너뛰기

    # ✅ 5-2. `qa`, `ner`, `nli`, `ned` 외의 task는 무시
    if task not in task_sample_limits:
        continue  # ✅ 필요 없는 task는 샘플링 대상에서 제외

    # ✅ 5-3. 샘플링 개수 확인 및 중복 방지
    sample_limit = task_sample_limits.get(task, default_sample_limit)
    
    if task_counts.get(task, 0) < sample_limit and input_output_pair not in train_pairs:
        if task not in task_sampled_data:
            task_sampled_data[task] = []
        task_sampled_data[task].append(row)
        task_counts[task] += 1

    # ✅ 5-4. 모든 task의 샘플링 개수가 충족되면 종료
    if all(task_counts.get(task, 0) >= task_sample_limits.get(task, default_sample_limit) for task in task_sample_limits):
        break

# ✅ 6. 샘플링 결과 출력 (각 task별 5개 샘플 확인)
for task, samples in task_sampled_data.items():
    print(f"Task: {task}, Sampled: {len(samples)}")
    for sample in samples[:5]:  
        print(sample)

# ✅ 7. 샘플링된 데이터 DatasetDict로 변환
sampled_dataset = DatasetDict({
    task: Dataset.from_pandas(pd.DataFrame(samples)) for task, samples in task_sampled_data.items()
})

# ✅ 8. 필요하면 저장 가능 (주석 해제)
# sampled_dataset.save_to_disk("sampled_test_dataset")


Task: qa_objective_2, Sampled: 50
{'dataset': 'medmcqa', 'split_data': 'test', 'task': 'qa_objective_2', 'instruction': 'Select the most appropriate answer for the given medical question from the provided options.', 'input': 'Dopamine agonist used in diabetes Please select one of the following: A) Metformin, B) Bromocriptine, C) Cabergoline, D) Vanadium salts.', 'output': 'B) Bromocriptine', 'text_length': 244, '__index_level_0__': 36490}
{'dataset': 'medmcqa', 'split_data': 'test', 'task': 'qa_objective_2', 'instruction': 'Select the most appropriate answer for the given medical question from the provided options.', 'input': 'A 58-year-old diabetic came with chest pain, ECG shows ST elevation in II, III and AVF leads. Inference would be Please select one of the following: A) Inferior wall MI, B) Anterior wall MI, C) Lateral wall MI, D) Posterior wall MI.', 'output': 'A) Inferior wall MI', 'text_length': 343, '__index_level_0__': 36501}
{'dataset': 'medmcqa', 'split_data': 'test', 'tas

In [18]:
import json
import re
from tqdm import tqdm
from transformers import TextStreamer

FastLanguageModel.for_inference(model)

output_file = "inference_results_test_v1.jsonl"


task_max_new_tokens = {
    "alternative_diet": 512,
    "daily_diets": 2048,
    # "dfh_info": 256,
    "generation": 8192,
    "ie": 32,
    "nli": 32,
    "qa_objective_1": 128,
    "qa_objective_2": 128,
    "qa_objective_3": 128,
    "re": 64,
    "summarization": 8192, 
}


# 1. `tqdm`을 사용하여 진행 상태 표시
total_samples = sum(len(samples) for samples in sampled_dataset.values())
with open(output_file, 'w') as f_out:
    with tqdm(total=total_samples, desc="Processing samples", unit="sample") as pbar:
        for task, sample in sampled_dataset.items():
            for samp in sample:
                # 데이터셋 이름에 따른 context length 설정
                task_name = samp.get("task", "")
                # max_new_tokens = 8192 if dataset_name in ["meddialog", "pubmed"] else 8
                max_new_tokens = task_max_new_tokens.get(task_name, 8)  # 기본값 8


                # 입력 토큰 생성
                inputs = tokenizer(
                    [
                        alpaca_prompt.format(
                            samp['instruction'],  # instruction
                            samp['input'],  # input
                            "",  # output
                        )
                    ], return_tensors="pt"
                ).to("cuda")

                # TextStreamer 설정
                text_streamer = TextStreamer(tokenizer)
                
                # 모델 생성 및 출력
                output_tensor = model.generate(
                    **inputs, 
                    max_new_tokens=max_new_tokens
                )
                model_output = tokenizer.decode(output_tensor[0], skip_special_tokens=True)

                # `### Response:` 뒤의 텍스트 추출
                response_text = None
                response_match = re.search(r"### Response:\s*(.+)", model_output, re.DOTALL)
                if response_match:
                    response_text = response_match.group(1).strip()
                else:
                    response_text = "No valid response found"

                # 모델 출력 결과를 samp에 추가
                samp['model_output'] = response_text

                # JSONL 형식으로 저장
                f_out.write(json.dumps(samp, ensure_ascii=False) + "\n")
                
                # tqdm 진행 상태 업데이트
                pbar.update(1)


Processing samples:  38%|███▊      | 190/500 [41:49<1:31:49, 17.77s/sample]  